In [ ]:
# !pip install --upgrade google-api-python-client

In [2]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import math 

PROJECT = 'qwiklabs-gcp-aebfb78fe0f1b1d1' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'sarah-bucket' # REPLACE WITH YOUR BUCKET NAME
REGION = 'us-central1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1
os.environ['TFVERSION'] = '1.7'  # Tensorflow version

SEQ_LEN = 50

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Define the project, Bucket and Region and set them.

In [3]:
# for bash
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION

In [4]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [5]:
! pwd
! ls

/content/datalab/energy_forcasing/Sarah
ARIMA.csv		 data		  train_cloud_model.ipynb
BIG_q.ipynb		 Energy_ts	  trained
build_local_model.ipynb  hyperparam.yaml  train_local_model.ipynb
Build_model_task.ipynb	 sarah_data.csv
CNN_W2P_model		 test.json


In [6]:
CSV_COLUMNS = ['prediction_date', 'wind_speed_100m', 
  'wind_direction_100m', 
  'temperature', 
  'air_density',
  'pressure', 'precipitation', 'wind_gust', 'radiation', 
  'wind_speed', 'wind_direction', 'price']
FEATURES = CSV_COLUMNS[1:len(CSV_COLUMNS) - 1]
LABEL = CSV_COLUMNS[0]

df5 = pd.read_csv('/content/datalab/energy_forcasing/Sarah/sarah_data.csv', header = None, names = CSV_COLUMNS)
df5.describe()

,wind_speed_100m,wind_direction_100m,temperature,air_density,pressure,precipitation,wind_gust,radiation,wind_speed,wind_direction,price
count,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000
mean,4.799901,189.582975,14.920355,1.132338,935.286695,0.117364,4.872509,225.250182,3.220986,189.082629,50.153506
std,1.964154,58.126639,9.626527,0.039333,6.105083,0.245561,2.428587,266.299815,1.319704,56.712796,12.448392
min,1.372222,47.883333,-6.044444,1.036111,913.800000,0.000000,1.194444,0.000000,0.855556,50.538889,4.000000
25%,3.534722,142.598611,7.540278,1.106111,932.034722,0.000000,3.284722,0.000000,2.294444,143.750000,43.000000
50%,4.477778,191.908333,14.213889,1.131667,934.761111,0.000000,4.327778,95.000000,2.961111,191.208333,50.890000
75%,5.544444,232.043056,21.062500,1.160556,937.805556,0.100000,5.627778,416.422222,3.850000,230.865278,59.950000
max,15.044444,329.544444,41.044444,1.229444,961.238889,1.700000,16.705556,954.288889,9.738889,327.711111,85.050000


In [7]:
df2 = pd.read_csv('data/CNN_W2P_model/train-1.csv', header = None)
df2.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.948333,0.942500,0.912333,0.500000,0.781000,0.654167,0.808333,0.724833,0.992833,0.668833,...,0.658333,0.500000,1.085000,0.953333,0.500000,0.846000,0.784667,0.751667,0.247333,1.001667
1,0.942500,0.912333,0.500000,0.781000,0.654167,0.808333,0.724833,0.992833,0.668833,0.750000,...,0.500000,1.085000,0.953333,0.500000,0.846000,0.784667,0.751667,0.247333,1.001667,1.008333
2,0.912333,0.500000,0.781000,0.654167,0.808333,0.724833,0.992833,0.668833,0.750000,0.830833,...,1.085000,0.953333,0.500000,0.846000,0.784667,0.751667,0.247333,1.001667,1.008333,0.790500
3,0.500000,0.781000,0.654167,0.808333,0.724833,0.992833,0.668833,0.750000,0.830833,0.834500,...,0.953333,0.500000,0.846000,0.784667,0.751667,0.247333,1.001667,1.008333,0.790500,1.124667
4,0.781000,0.654167,0.808333,0.724833,0.992833,0.668833,0.750000,0.830833,0.834500,0.816667,...,0.500000,0.846000,0.784667,0.751667,0.247333,1.001667,1.008333,0.790500,1.124667,0.492167


In [8]:
df3 = pd.read_csv('data/CNN_W2P_model/valid-1.csv', header = None) #, names = CSV_COLUMNS)
df3.describe()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
count,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,...,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000
mean,0.827618,0.827851,0.827685,0.827717,0.827745,0.827840,0.827769,0.827687,0.827698,0.827717,...,0.829831,0.829818,0.829786,0.829823,0.829786,0.829683,0.829584,0.829590,0.829577,0.829446
std,0.217746,0.217454,0.217529,0.217534,0.217526,0.217463,0.217445,0.217395,0.217400,0.217403,...,0.215509,0.215509,0.215515,0.215481,0.215470,0.215397,0.215381,0.215381,0.215382,0.215333
min,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,...,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667
25%,0.707250,0.707542,0.707250,0.707250,0.707250,0.707542,0.707542,0.707542,0.707542,0.707542,...,0.710625,0.710625,0.710625,0.710625,0.710625,0.710625,0.710625,0.710625,0.710625,0.710625
50%,0.842167,0.842833,0.842167,0.842833,0.842833,0.842833,0.842167,0.842167,0.842167,0.842833,...,0.844917,0.844833,0.844833,0.844833,0.844833,0.844833,0.844167,0.844833,0.844833,0.844167
75%,0.995667,0.995667,0.995667,0.995667,0.995667,0.995667,0.995667,0.995125,0.995125,0.995125,...,0.995000,0.995000,0.995000,0.995000,0.995000,0.994958,0.994833,0.994833,0.994833,0.994833
max,1.417500,1.417500,1.417500,1.417500,1.417500,1.417500,1.417500,1.417500,1.417500,1.417500,...,1.417500,1.417500,1.417500,1.417500,1.417500,1.417500,1.417500,1.417500,1.417500,1.417500


This is where youre model lives and can be pushed out to the cloud.

In [9]:
! ls /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer

__init__.py  __init__.pyc  model.py  model.pyc	task.py


In [10]:
ls /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trained/cnn/export/exporter/

1537820821/


### Push out the training and evaluation data for the cloud
You can check to make sure the files were moved by looking in the "bucket" or "storage" and looking for the data files (.csv's in our case).

In [ ]:
import shutil
shutil.rmtree('/data/CNN_W2P_model ', ignore_errors=True)
os.makedirs('/data/CNN_W2P_model/')

In [ ]:
%bash
gsutil -m rm -rf gs://${BUCKET}/CNN_W2P_model/*
gsutil -m cp data/CNN_W2P_model/*.csv gs://${BUCKET}/CNN_W2P_model

### Push out to the cloud
After you run the code below - you can watch the process kick off and run on "ML Engine"

In [ ]:
%%bash
for MODEL in linear dnn cnn rnn rnn2  ; do # rnnN
  OUTDIR=gs://${BUCKET}/CNN_W2P_model/${MODEL}
  JOBNAME=CNN_W2P_model_${MODEL}$(date -u +%y%m%d_%H%M%S)
  SEQ_LEN=50
  REGION=us-central1
  gsutil -m rm -rf $OUTDIR
  echo $JOBNAME
  gcloud ml-engine jobs submit training $JOBNAME \
     --region=$REGION \
     --module-name=trainer.task \
     --package-path=${PWD}/CNN_W2P_model/trainer \
     --job-dir=$OUTDIR \
     --scale-tier=PREMIUM_1 \
     --runtime-version=$TFVERSION \
     -- \
     --train_data_path="gs://${BUCKET}/CNN_W2P_model/train-1.csv" \
     --eval_data_path="gs://${BUCKET}/CNN_W2P_model/valid-1.csv"  \
     --output_dir=$OUTDIR \
     --train_steps=2000 --sequence_length=$SEQ_LEN --model=$MODEL
done


### Tensor Boards are finky and not always willing to run.  You can try in the window - or go back to the cloud shell and run the "command line access for TB."

In [ ]:
from google.datalab.ml import TensorBoard
TensorBoard().start('gs://{}/CNN_W2P_model'.format(BUCKET))

In [ ]:
for pid in TensorBoard.list()['pid']:
  TensorBoard().stop(pid)
  print 'Stopped TensorBoard with pid {}'.format(pid)

In [ ]:
## command line access of the TB
print('tensorboard --logdir=gs://{}/CNN_W2P_model --port=8083'.format(BUCKET))

## Deploy the model

### Convert the data into something that can be read into the model, so strings that are 49 long where the 50th price is perdicted.

In [19]:
MODEL_NAME="CNN_W2P_model" ## you can name it w/e you want
MODEL_VERSION="ml_on_gcp" ## you can name it w/e you want
MODEL_TYPE = "linear"
MODEL_LOCATION="$(gsutil ls gs://${}/{}/{}/export/exporter/| tail -1)".format(BUCKET,MODEL_NAME,MODEL_TYPE)

In [20]:
os.environ['MODEL_NAME'] = MODEL_NAME
os.environ['MODEL_VERSION'] = MODEL_VERSION
os.environ['MODEL_TYPE'] = MODEL_TYPE
os.environ['MODEL_LOCATION'] = MODEL_LOCATION

In [21]:
# %bash
# gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}

This will delete version [ml_on_gcp]...

Do you want to continue (Y/n)?  
ERROR: (gcloud.ml-engine.versions.delete) FAILED_PRECONDITION: Field: name Error: Cannot delete the default version unless it is the only version in the model.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: Cannot delete the default version unless it is the only version in
      the model.
    field: name


In [ ]:
# %bash
# gcloud ml-engine models delete ${MODEL_NAME}

In [ ]:
%bash
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version $TFVERSION --enable-logging

In [22]:
print(MODEL_LOCATION)

$(gsutil ls gs://$sarah-bucket/energy_price_forecast/linear/export/exporter/| tail -1)


Make data to feed to model

In [31]:
def to_csv(filename, start, N, J):
  with open(filename, 'w') as ofp:
    for ii in range(start, N):
      # nn = int(round(np.random.uniform(low = 0, high = 6466-J),0))
      seq = (df5.price[(ii):(ii+J)]/60) ## trade nn for ii to make an in seq dataset instead of random
      line = ",".join(map(str, seq))
      ofp.write(line + '\n')
      
      if len(seq) != J:
        break

import os
try:
  os.makedirs('data/CNN_W2P_model/')
except OSError:
  pass

to_csv('data/CNN_W2P_model/to_model', start = 0, N = 6596, J = 50) #  (6646 - 50) = 6596

In [33]:
model_data = pd.read_csv('/content/datalab/energy_forcasing/Sarah/data/CNN_W2P_model/to_model.csv', header = None)
model_data.shape

(6597, 49)

In [ ]:
!pwd
!ls

### To get the view on the model API go to ML Engine > Models > Model Details 

In [27]:
%%writefile test.json
{"price": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}

Overwriting test.json


In [24]:
%%writefile test.json
[45.52, 45.24, 43.792, 24.0, 37.488, 31.400000000000002, 38.800000000000004, 34.792, 47.656000000000006, 32.104000000000006, 36.0, 39.88, 40.056000000000004, 39.2, 34.952, 50.42400000000001, 36.472, 42.848000000000006, 30.496, 44.712, 32.128, 52.08, 31.664, 47.88, 44.0, 51.352000000000004, 42.624, 54.879999999999995, 32.800000000000004, 31.400000000000002, 30.680000000000003, 44.608000000000004, 32.256, 50.072, 37.424, 39.88, 47.752, 44.800000000000004, 52.08, 46.952, 31.6, 24.0, 52.08, 45.760000000000005, 24.0, 40.608000000000004, 37.664, 36.080000000000005, 11.872]

Overwriting test.json


In [28]:
%bash
gcloud ml-engine predict --model CNN_W2P_model  \
                   --version ml_on_gcp \
                   --json-instances test.json

PREDICTED
[0.0]


In [29]:
from oauth2client.client import GoogleCredentials
import requests
import json

token = GoogleCredentials.get_application_default().get_access_token().access_token
#print(token)
api = 'https://ml.googleapis.com/v1/projects/{}/models/{}/versions/{}:predict' \
         .format(PROJECT, MODEL_NAME, MODEL_VERSION)

headers = {'Authorization': 'Bearer ' + token }
data = {
  'instances': [
{'price': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}
  ]
}
response = requests.post(api, json=data, headers=headers)
print(response.content)

b'{"predictions": [{"predicted": [0.0]}]}'


This will take a few minutes.

In [ ]:
result = []
for y in range(1,model_data.shape[0]):
    raw_prices = model_data.iloc[y]
    prices = [float(price) for price in raw_prices]
    #print prices
    data = {
      'instances': [
        {
          'price': prices
        },
      ]
    }
    #print(data)
    response = requests.post(api, json=data, headers=headers)
    #print y/model_data.shape[0]
    forecast = response.json()['predictions'][0]['predicted'][0] ## (df5.price[(nn):(nn+J)])/60 ##response.content[32:-4]
    #print (forecast)
    result.append(response.content[32:-4])